# United Nations Security Council Consolidated List V3

https://www.un.org/securitycouncil/content/un-sc-consolidated-list


<td>
    <a target="_blank" href=" https://colab.research.google.com/drive/1um08RzU0GotlbbUP8XGu7lcd_r3CsL6v?usp=sharing"> run on Google Colab</a>
  <td> 🤗

In [ ]:
!pip install xmltodict
import xmltodict
import json
import pandas as pd
from tqdm.notebook import tqdm
from google.colab import data_table

data_table.enable_dataframe_formatter()
data_table.disable_dataframe_formatter()
# download file in xml format
!wget https://scsanctions.un.org/resources/xml/en/consolidated.xml

with open("consolidated.xml") as xml_file:    
    data_dict = xmltodict.parse(xml_file.read())
    xml_file.close()
    json_data = json.dumps(data_dict)
    with open("data.json", "w") as json_file:
        json_file.write(json_data)
        json_file.close()
        
df = pd.read_json('data.json')

dict_ENTITIES = df.CONSOLIDATED_LIST.ENTITIES
df_ENTITIES = pd.DataFrame.from_dict(dict_ENTITIES)

dict_INDIVIDUALS = df.CONSOLIDATED_LIST.INDIVIDUALS
df_INDIVIDUALS = pd.DataFrame.from_dict(dict_INDIVIDUALS)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-05-30 14:11:10--  https://scsanctions.un.org/resources/xml/en/consolidated.xml
Resolving scsanctions.un.org (scsanctions.un.org)... 157.150.185.69
Connecting to scsanctions.un.org (scsanctions.un.org)|157.150.185.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1583269 (1.5M) [text/xml]
Saving to: ‘consolidated.xml.2’

consolidated.xml.2  100%[===================>]   1.51M  9.72MB/s    in 0.2s    

2022-05-30 14:11:11 (9.72 MB/s) - ‘consolidated.xml.2’ saved [1583269/1583269]



In [ ]:
df_INDIVIDUALS

,INDIVIDUAL
0,"{'DATAID': '6908555', 'VERSIONNUM': '1', 'FIRS..."
1,"{'DATAID': '6908570', 'VERSIONNUM': '1', 'FIRS..."
2,"{'DATAID': '6908571', 'VERSIONNUM': '1', 'FIRS..."
3,"{'DATAID': '6908858', 'VERSIONNUM': '1', 'FIRS..."
4,"{'DATAID': '6908565', 'VERSIONNUM': '1', 'FIRS..."
...,...
695,"{'DATAID': '6908704', 'VERSIONNUM': '1', 'FIRS..."
696,"{'DATAID': '6908707', 'VERSIONNUM': '1', 'FIRS..."
697,"{'DATAID': '6908841', 'VERSIONNUM': '1', 'FIRS..."
698,"{'DATAID': '2975510', 'VERSIONNUM': '1', 'FIRS..."


# 1 Clean ENTITIES List

In [ ]:
def dict_2_df_mini(dataset,col_name):

    df =pd.DataFrame()
     
    for i in range(len(dataset)):

        df_temp = pd.DataFrame.from_dict(dataset[col_name][i], orient='index').T
        df = df.append(df_temp,ignore_index=True)
        # df = df.rename(index=dataset.index)
        # drop original col containing dictionary with two keys
  
    new_data = pd.concat([dataset,df],axis=1)
    new_data = new_data.drop([col_name],axis=1)

    return new_data

df_ENTITIES_all = dict_2_df_mini(df_ENTITIES,'ENTITY')

In [ ]:
df_ENTITIES_all.head(2)

,DATAID,VERSIONNUM,FIRST_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,LAST_DAY_UPDATED,ENTITY_ALIAS,ENTITY_ADDRESS,SORT_KEY,SORT_KEY_LAST_MOD,NAME_ORIGINAL_SCRIPT,SUBMITTED_ON
0,6908629,1,PROPAGANDA AND AGITATION DEPARTMENT (PAD),DPRK,KPe.053,2017-09-11,The Propaganda and Agitation Department has fu...,{'VALUE': 'UN List'},{'VALUE': None},"{'QUALITY': None, 'ALIAS_NAME': None}","{'CITY': 'Pyongyang', 'COUNTRY': 'Democratic P...",None,None,NaN,NaN
1,110403,1,7TH OF TIR,Iran,IRe.001,2006-12-23,Subordinate of Defence Industries Organisation...,{'VALUE': 'UN List'},{'VALUE': '2014-12-17'},"{'QUALITY': None, 'ALIAS_NAME': None}",None,None,None,NaN,NaN


## 1.1 ENTITY_ADDRESS

In [ ]:
df_ENTITY_ADDRESS = pd.DataFrame(columns=['STREET','CITY','STATE_PROVINCE','COUNTRY','NOTE'])
dict_temp = pd.DataFrame()
list_temp = pd.DataFrame()
none_index = []
dict_index =[]
list_index =[]
for index, row in df_ENTITIES_all.iterrows():
  row = row['ENTITY_ADDRESS']
  if row == None:
      # print(index,row)
      df_ENTITY_ADDRESS = df_ENTITY_ADDRESS.append({'STREET':None,'CITY':None,\
                                                    'STATE_PROVINCE':None,'COUNTRY':None,\
                                                    'NOTE':None},ignore_index=True) 
      none_index.append(index)
      df_ENTITY_ADDRESS.index= none_index
  else:
    if type(row)==dict:
      df_temp = pd.DataFrame.from_dict(row, orient="index").T
      dict_temp = dict_temp.append(df_temp,ignore_index=True)

      dict_index.append(index)
      dict_temp.index=dict_index


    elif type(row)==list:
      df_temp = pd.DataFrame.from_dict(row[0], orient="index").T
      list_temp = list_temp.append(df_temp,ignore_index=True)

      list_index.append(index)
      list_temp.index=list_index


df_ENTITY_ADDRESS = pd.concat([df_ENTITY_ADDRESS,dict_temp,list_temp]) 
df_ENTITY_ADDRESS.sort_index(inplace=True) 

In [ ]:
df_ENTITY_ADDRESS

,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE
0,NaN,Pyongyang,NaN,Democratic People's Republic of Korea,NaN,NaN
1,NaN,None,NaN,None,None,NaN
2,NaN,NaN,NaN,NaN,"Operates in Lebanon, Syria and the Arabian Pen...",NaN
3,NaN,NaN,NaN,Philippines,NaN,NaN
4,NaN,None,NaN,None,None,NaN
...,...,...,...,...,...,...
248,"419-201, Tongyi Lu, Huancui Qu, Weihai",Shandong,NaN,China,NaN,264200
249,None,None,NaN,None,NaN,NaN
250,"Mehrabad International Airport, Next to Termin...",Tehran,NaN,Iran (Islamic Republic of),NaN,NaN
251,"Pasdaran Avenue, next to Telecommunication Ind...","Tehran, 16588",NaN,Iran,NaN,NaN


## 1.2 ENTITY LIST TYPE, LAST DAY UPDATED, ENTITY _ALIAS

In [ ]:
def Dict_2_df_Large(dataset,col_name):
    
    if (col_name == 'LIST_TYPE' and len(dataset.LIST_TYPE[1])==1):

        df_LIST_TYPE = pd.DataFrame(columns=['LIST_TYPE'])
        for index, row in dataset.iterrows():
          row = row[col_name]
          value = "".join(row.values())
          df_LIST_TYPE = df_LIST_TYPE.append({'LIST_TYPE':value},ignore_index=True)

        return df_LIST_TYPE

    if (col_name == 'LAST_DAY_UPDATED' and len(dataset.LAST_DAY_UPDATED[1])==1):

        df_LAST_DAY_UPDATED = pd.DataFrame(columns=['LAST_DAY_UPDATED'])
        for index, row in dataset.iterrows():
          row = row[col_name]
          value = "".join(row.values())
          df_LAST_DAY_UPDATED = df_LAST_DAY_UPDATED.append({'LAST_DAY_UPDATED':value},ignore_index=True)
        
        return df_LAST_DAY_UPDATED

    elif col_name == 'ENTITY_ALIAS':

      df_ENTITY_ALIAS = pd.DataFrame(columns=['QUALITY','ALIAS_NAME'])

      for index, row in dataset.iterrows():
        row = row[col_name]
        if type(row) == float:
          df_ENTITY_ALIAS = df_ENTITY_ALIAS.append({'QUALITY':None,'ALIAS_NAME':None},ignore_index=True)
        else:
          n = len(row)
          if n > 2:
            df_temp = pd.DataFrame(row)
            df_temp = df_temp.groupby([df_temp.columns[0]])[df_temp.columns[1]].apply(lambda x: ','.join(x.values))
            df_temp = pd.DataFrame(df_temp)
            df_temp.reset_index(inplace=True)
            df_ENTITY_ALIAS = df_ENTITY_ALIAS.append({'QUALITY':df_temp.QUALITY[0],'ALIAS_NAME':df_temp.ALIAS_NAME[0]},ignore_index=True)
          elif n==2:
            df_temp = pd.DataFrame.from_dict(df_ENTITIES_all['ENTITY_ALIAS'][0], orient="index").T
            df_ENTITY_ALIAS = df_ENTITY_ALIAS.append({'QUALITY':df_temp.QUALITY[0],'ALIAS_NAME':df_temp.ALIAS_NAME[0]},ignore_index=True) 

      return df_ENTITY_ALIAS

    elif col_name == 'ENTITY_ADDRESS':

        df_ENTITY_ADDRESS = pd.DataFrame(columns=['STREET','CITY','STATE_PROVINCE','COUNTRY','NOTE'])
        dict_temp = pd.DataFrame()
        list_temp = pd.DataFrame()
        none_index = []
        dict_index =[]
        list_index =[]
        for index, row in df_ENTITIES_all.iterrows():
          row = row['ENTITY_ADDRESS']
          if row == None:
              # print(index,row)
              df_ENTITY_ADDRESS = df_ENTITY_ADDRESS.append({'STREET':None,'CITY':None,\
                                                            'STATE_PROVINCE':None,'COUNTRY':None,\
                                                            'NOTE':None},ignore_index=True) 
              none_index.append(index)
              df_ENTITY_ADDRESS.index= none_index
          else:
            if type(row)==dict:
              df_temp = pd.DataFrame.from_dict(row, orient="index").T
              dict_temp = dict_temp.append(df_temp,ignore_index=True)

              dict_index.append(index)
              dict_temp.index=dict_index

            elif type(row)==list:
              df_temp = pd.DataFrame.from_dict(row[0], orient="index").T
              list_temp = list_temp.append(df_temp,ignore_index=True)

              list_index.append(index)
              list_temp.index=list_index

        df_ENTITY_ADDRESS = pd.concat([df_ENTITY_ADDRESS,dict_temp,list_temp]) 
        df_ENTITY_ADDRESS.sort_index(inplace=True) 

        return df_ENTITY_ADDRESS

In [ ]:
df_ENTITY_ALIAS = Dict_2_df_Large(df_ENTITIES_all,'ENTITY_ALIAS')

In [ ]:
df_LIST_TYPE = Dict_2_df_Large(df_ENTITIES_all,'LIST_TYPE')

In [ ]:
df_ENTITY_ADDRESS = Dict_2_df_Large(df_ENTITIES_all,'ENTITY_ADDRESS')

In [ ]:
df_ENTITY_ADDRESS

,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE
0,NaN,Pyongyang,NaN,Democratic People's Republic of Korea,NaN,NaN
1,NaN,None,NaN,None,None,NaN
2,NaN,NaN,NaN,NaN,"Operates in Lebanon, Syria and the Arabian Pen...",NaN
3,NaN,NaN,NaN,Philippines,NaN,NaN
4,NaN,None,NaN,None,None,NaN
...,...,...,...,...,...,...
248,"419-201, Tongyi Lu, Huancui Qu, Weihai",Shandong,NaN,China,NaN,264200
249,None,None,NaN,None,NaN,NaN
250,"Mehrabad International Airport, Next to Termin...",Tehran,NaN,Iran (Islamic Republic of),NaN,NaN
251,"Pasdaran Avenue, next to Telecommunication Ind...","Tehran, 16588",NaN,Iran,NaN,NaN


In [ ]:
df = pd.concat([df_ENTITIES_all,df_ENTITY_ALIAS,df_LIST_TYPE,df_ENTITY_ADDRESS],axis=1)

In [ ]:
df.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,LAST_DAY_UPDATED,ENTITY_ALIAS,...,SUBMITTED_ON,QUALITY,ALIAS_NAME,LIST_TYPE,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE
0,6908629,1,PROPAGANDA AND AGITATION DEPARTMENT (PAD),DPRK,KPe.053,2017-09-11,The Propaganda and Agitation Department has fu...,{'VALUE': 'UN List'},{'VALUE': None},"{'QUALITY': None, 'ALIAS_NAME': None}",...,NaN,None,None,UN List,NaN,Pyongyang,NaN,Democratic People's Republic of Korea,NaN,NaN


In [ ]:
df = df.drop(['LIST_TYPE','ENTITY_ALIAS','ENTITY_ADDRESS'],axis=1)

## 1.3 LAST DAY UPDATED

In [ ]:
df['LAST_DAY_UPDATED'] = df['LAST_DAY_UPDATED'].apply(lambda x: str(x.values()).replace("dict_values(['","").replace("'])",""))

In [ ]:
df['LAST_DAY_UPDATED'] = df['LAST_DAY_UPDATED'].apply(lambda x: str(x).replace("dict_values([","").replace("])",""))

In [ ]:
df['LAST_DAY_UPDATED'] = df['LAST_DAY_UPDATED'].apply(lambda x: x.replace("[","").replace("]",""))

In [ ]:
df.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LAST_DAY_UPDATED,SORT_KEY,SORT_KEY_LAST_MOD,NAME_ORIGINAL_SCRIPT,SUBMITTED_ON,QUALITY,ALIAS_NAME,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE
0,6908629,1,PROPAGANDA AND AGITATION DEPARTMENT (PAD),DPRK,KPe.053,2017-09-11,The Propaganda and Agitation Department has fu...,None,None,None,NaN,NaN,None,None,NaN,Pyongyang,NaN,Democratic People's Republic of Korea,NaN,NaN


In [ ]:
# save to local
df.to_csv('UN_Sanctioned_ENTITIES.csv')

# 2 Clean INDIVIDUALS List

In [ ]:
# df_individuals_all = pd.DataFrame()
# for i in tqdm(range(len(df_INDIVIDUALS))):
#     df_temp = pd.DataFrame.from_dict(df_INDIVIDUALS.INDIVIDUAL[i], orient='index').T
#     df_individuals_all = df_individuals_all.append(df_temp,ignore_index=True)

## 2.1 Clean INDIVIDUAL_ADDRESS column

In [ ]:
df_individuals_all = dict_2_df_mini(df_INDIVIDUALS,'INDIVIDUAL')

In [ ]:
df_INDIVIDUAL_ADDRESS = pd.DataFrame(columns=['STREET','CITY','STATE_PROVINCE','COUNTRY','NOTE'])
dict_temp = pd.DataFrame()
list_temp = pd.DataFrame()
none_index = []
dict_index =[]
list_index =[]

for index, row in df_individuals_all.iterrows():
  row = row['INDIVIDUAL_ADDRESS']
  if row == None:
      # print(index,row)
      df_INDIVIDUAL_ADDRESS = df_INDIVIDUAL_ADDRESS.append({'STREET':None,'CITY':None,\
                                                    'STATE_PROVINCE':None,'COUNTRY':None,\
                                                    'NOTE':None},ignore_index=True) 
      none_index.append(index)
      
  else:
    if type(row)==dict:
      df_temp = pd.DataFrame.from_dict(row, orient="index").T
      dict_temp = dict_temp.append(df_temp,ignore_index=True)

      dict_index.append(index)
      

    elif type(row)==list:
      df_temp = pd.DataFrame.from_dict(row[0], orient="index").T
      list_temp = list_temp.append(df_temp,ignore_index=True)

      list_index.append(index)
      

df_INDIVIDUAL_ADDRESS.index= none_index
dict_temp.index=dict_index
list_temp.index=list_index

INDIVIDUAL_ADDRESS = pd.concat([df_INDIVIDUAL_ADDRESS,dict_temp,list_temp])
df_individuals_all = pd.concat([df_individuals_all,INDIVIDUAL_ADDRESS],axis=1)
df_individuals_all = df_individuals_all.drop(['INDIVIDUAL_ADDRESS'],axis=1)

In [ ]:
df_individuals_all.head(2) 

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,DESIGNATION,...,FOURTH_NAME,GENDER,TITLE,SUBMITTED_BY,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,{'VALUE': 'DPRK Ministry of State Security Off...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,6908570,1,CHANG,CHANG HA,None,DPRK,KPi.037,2016-11-30,None,{'VALUE': 'President of the Second Academy of ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [ ]:
list(df_individuals_all.columns)

['DATAID',
 'VERSIONNUM',
 'FIRST_NAME',
 'SECOND_NAME',
 'THIRD_NAME',
 'UN_LIST_TYPE',
 'REFERENCE_NUMBER',
 'LISTED_ON',
 'COMMENTS1',
 'DESIGNATION',
 'NATIONALITY',
 'LIST_TYPE',
 'LAST_DAY_UPDATED',
 'INDIVIDUAL_ALIAS',
 'INDIVIDUAL_DATE_OF_BIRTH',
 'INDIVIDUAL_PLACE_OF_BIRTH',
 'INDIVIDUAL_DOCUMENT',
 'SORT_KEY',
 'SORT_KEY_LAST_MOD',
 'NAME_ORIGINAL_SCRIPT',
 'FOURTH_NAME',
 'GENDER',
 'TITLE',
 'SUBMITTED_BY',
 'STREET',
 'CITY',
 'STATE_PROVINCE',
 'COUNTRY',
 'NOTE',
 'ZIP_CODE']

## 2.2 Clean DESIGNATION 

In [ ]:
pd.DataFrame(df_individuals_all.DESIGNATION)

,DESIGNATION
0,{'VALUE': 'DPRK Ministry of State Security Off...
1,{'VALUE': 'President of the Second Academy of ...
2,{'VALUE': 'Chairman of the Second Economic Com...
3,NaN
4,{'VALUE': 'DPRK Ministry of State Security Off...
...,...
695,{'VALUE': 'Commander of the Anas al-Dabbashi m...
696,{'VALUE': 'Commander of the Coast Guard in Zaw...
697,NaN
698,NaN


In [ ]:
df_DESIGNATION = pd.DataFrame(columns=['DESIGNATION'])
dict_temp = pd.DataFrame()
list_temp = pd.DataFrame()
none_index = []
dict_index =[]
list_index =[]

for index,row in df_individuals_all.iterrows():
  row = row['DESIGNATION']
  if type(row) == float:
    df_DESIGNATION = df_DESIGNATION.append({'DESIGNATION':None},ignore_index=True)

    none_index.append(index)
    
    
  elif type(row) == dict:
    # if len(row)==1:
      df_temp = pd.DataFrame.from_dict(row, orient="index").T
      if len(df_temp)>1:
        # df_temp_copy = df_temp.copy(deep=True)
        x= df_temp
        x['V'] = 'V'
        df_temp = x.groupby(['V'])['VALUE'].apply(lambda x:', '.join(x) )
        
        list_temp = list_temp.append(df_temp, ignore_index=True)
        # 
        list_index.append(index)
        list_temp.index=list_index
        
        # print('bigger 1,{index}'.format(index=index))

      else:     
        dict_temp = dict_temp.append(df_temp, ignore_index=True)
        dict_index.append(index)

df_DESIGNATION.index= none_index        
list_temp.index=list_index
dict_temp.index=dict_index
list_temp.rename(columns={'V':'DESIGNATION'},inplace=True) 
dict_temp.rename(columns={'VALUE':'DESIGNATION'},inplace=True)     
  # else:
  #   print('other type')

DESIGNATION = pd.concat([df_DESIGNATION,list_temp,dict_temp])
DESIGNATION.sort_index(inplace=True) 
df_individuals_all = df_individuals_all.drop(['DESIGNATION'],axis=1)
df_individuals_all = pd.concat([df_individuals_all,DESIGNATION],axis=1)

In [ ]:
df_individuals_all.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,NATIONALITY,...,GENDER,TITLE,SUBMITTED_BY,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE,DESIGNATION
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,{'VALUE': 'Democratic People's Republic of Kor...,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,DPRK Ministry of State Security Official


## 2.3 Clean NATIONALITY

In [ ]:
pd.DataFrame(df_individuals_all.NATIONALITY).to_csv('NATIONALITY.csv')

In [ ]:
df_NATIONALITY = pd.DataFrame(columns=['NATIONALITY'])
list_temp = pd.DataFrame()
dict_temp = pd.DataFrame()
list_index = []
dict_index = []
none_index = []

for index, row in df_individuals_all.iterrows():
  row = row['NATIONALITY']
  if type(row) == float:
    df_NATIONALITY = df_NATIONALITY.append({'NATIONALITY':None},ignore_index=True)
    none_index.append(index)
    # print('float:{index}'.format(index=index))

  elif type(row) == dict:
    df_temp = pd.DataFrame.from_dict(row, orient="index").T
    #print('dict:{index}'.format(index=index))
    if len(df_temp)>1:
      # df_temp_copy = df_temp.copy(deep=True)
      x= df_temp
      x['V'] = 'V'
      df_temp = x.groupby(['V'])['VALUE'].apply(lambda x:', '.join(x) )
      
      list_temp = list_temp.append(df_temp, ignore_index=True)
  #     # 
      list_index.append(index)

  #   # print('bigger 1,{index}'.format(index=index))

    else:     
      dict_temp = dict_temp.append(df_temp, ignore_index=True)
      dict_index.append(index)
  # elif type(row) == int:
  #   print('int:{index}'.format(index=index))

  else:
    print('ERROR: row value type is {t}, index is {index}'.format(t=type(row),index=index))

if len(list_temp) + len(df_NATIONALITY) + len(dict_temp) == len(df_individuals_all):
  print('Lenth Matches')

  # list_temp.index = list_index
  # df_NATIONALITY.index = none_index
  # dict_temp.index =  dict_index

  df_NATIONALITY.index= none_index        
  list_temp.index=list_index
  dict_temp.index=dict_index

  list_temp.rename(columns={'V':'NATIONALITY'},inplace=True)    
  dict_temp.rename(columns={'VALUE':'NATIONALITY'},inplace=True) 

  # NATIONALITY = pd.concat([list_temp,df_NATIONALITY,list_temp]).sort_index()

else:
  print('ERROR: Lenth MisMatches')

NATIONALITY = pd.concat([dict_temp,list_temp,df_NATIONALITY]).sort_index()
NATIONALITY

Lenth Matches


,NATIONALITY
0,Democratic People's Republic of Korea
1,Democratic People's Republic of Korea
2,Democratic People's Republic of Korea
3,"Trinidad and Tobago, United States of America"
4,Democratic People's Republic of Korea
...,...
695,Libya
696,Libya
697,Iraq
698,Afghanistan


In [ ]:
df_individuals_all = df_individuals_all.drop(['NATIONALITY'],axis=1)
df_individuals_all = pd.concat([df_individuals_all,NATIONALITY],axis=1)

In [ ]:
df_individuals_all.head(2)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,TITLE,SUBMITTED_BY,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE,DESIGNATION,NATIONALITY
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,{'VALUE': 'UN List'},...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,DPRK Ministry of State Security Official,Democratic People's Republic of Korea
1,6908570,1,CHANG,CHANG HA,None,DPRK,KPi.037,2016-11-30,None,{'VALUE': 'UN List'},...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,President of the Second Academy of Natural Sci...,Democratic People's Republic of Korea


## 2.4 Clean LIST TYPE

In [ ]:
df_individuals_all['LIST_TYPE'] = df_individuals_all['LIST_TYPE'].apply(lambda x: str(x.values()).replace("dict_values(['","").replace("'])",""))

In [ ]:
df_individuals_all.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,TITLE,SUBMITTED_BY,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE,DESIGNATION,NATIONALITY
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,UN List,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,DPRK Ministry of State Security Official,Democratic People's Republic of Korea


## 2.5 Clean LAST_DAY_UPDATED

In [ ]:
df_individuals_all['LAST_DAY_UPDATED']= df_individuals_all['LAST_DAY_UPDATED'].apply(lambda x: str(x.values()).\
                                                                                     replace("dict_values([","").replace("])","").\
                                                                                     replace("[","").replace("]",""))

In [ ]:
df_individuals_all.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,TITLE,SUBMITTED_BY,STREET,CITY,STATE_PROVINCE,COUNTRY,NOTE,ZIP_CODE,DESIGNATION,NATIONALITY
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,UN List,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,DPRK Ministry of State Security Official,Democratic People's Republic of Korea


In [ ]:
# def dict_2_df_indiv(dataset,col_name):

#   df =pd.DataFrame()
#   # if number of key is 1
#   if len(dataset[col_name][0].keys()) == 1:
#          dataset[col_name] = dataset[col_name].apply(lambda x: str(x.values()).replace("dict_values(['","").replace("'])",""))

#   # if number of key is >=1       
#   else:
#     for i in range(len(dataset)):

#       if len(dataset[col_name][i])>2:
#           df_temp = pd.DataFrame(dataset[col_name][i])
#           df_temp.set_index(df.columns[0],inplace=True)
#           df_temp = df_temp.groupby([df_temp.columns[0]])[df_temp.columns[0]].apply(lambda x: ','.join(x.values))
#           df_temp = pd.DataFrame(df_temp)
#           df = df.append(df_temp,ignore_index=True)
          
#       else:

#           df_temp = pd.DataFrame.from_dict(dataset[col_name][i], orient='index').T
#           df = df.append(df_temp,ignore_index=True)
#           # df = df.rename(index=dataset.index)
#           # drop original col containing dictionary with two keys
  
#   new_data = pd.concat([dataset,df],axis=1)
#   new_data = new_data.drop([col_name],axis=1)


#   # rename column
#   if col_name == 'INDIVIDUAL_DATE_OF_BIRTH':
#     new_data.rename(columns={'DATE':'DATE_OF_BIRTH'},inplace=True)

#   elif col_name == 'INDIVIDUAL_DOCUMENT':
#     new_data.rename(columns={'NUMBER':'DOCUMENT_NUMBER'},inplace=True)

#   return new_data


# # specify columns which contains dictionaries
# indiv_col = ['NATIONALITY','LIST_TYPE','LAST_DAY_UPDATED','INDIVIDUAL_ALIAS','INDIVIDUAL_ADDRESS','INDIVIDUAL_DATE_OF_BIRTH','INDIVIDUAL_DOCUMENT']
# for i in indiv_col:
#     df_indiv_all = dict_2_df_indiv(df_individuals_all, i)


## 2.6 Clean INDIVIDUAL_DATE_OF_BIRTH

In [ ]:

list_temp = pd.DataFrame()
dict_temp = pd.DataFrame(columns=['TYPE_OF_DATE','DATE_OF_BIRTH'])
list_index = []
dict_index = []

# type_list = []
for index, row in df_individuals_all.iterrows():
  row = row['INDIVIDUAL_DATE_OF_BIRTH']
  # type_list.append(type(row))
  # print(pd.DataFrame(type_list).rename(columns={0:'type_'}).type_.unique())
  if type(row) == dict:
    temp = pd.DataFrame.from_dict(row, orient="index").T
    dict_temp = dict_temp.append(temp,ignore_index=True)

    dict_index.append(index)
    # print('float:{index}'.format(index=index))

  if type(row) == list:
    temp = pd.DataFrame.from_dict(row[0], orient="index").T
    list_temp = list_temp.append(temp,ignore_index=True)
    list_index.append(index)

list_temp.index=list_index
dict_temp.index=dict_index

In [ ]:
INDIVIDUAL_DATE_OF_BIRTH = pd.concat([list_temp,dict_temp]).sort_index()
INDIVIDUAL_DATE_OF_BIRTH

,TYPE_OF_DATE,DATE,YEAR,NOTE,FROM_YEAR,TO_YEAR,DATE_OF_BIRTH
0,EXACT,1964-07-17,NaN,NaN,NaN,NaN,NaN
1,EXACT,1964-01-10,NaN,NaN,NaN,NaN,NaN
2,EXACT,1960-04-04,NaN,NaN,NaN,NaN,NaN
3,EXACT,1967-07-04,NaN,NaN,NaN,NaN,NaN
4,EXACT,1973-09-30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
695,EXACT,1988-05-07,NaN,NaN,NaN,NaN,NaN
696,EXACT,1986-07-27,NaN,NaN,NaN,NaN,NaN
697,EXACT,1976-10-05,NaN,NaN,NaN,NaN,NaN
698,EXACT,NaN,1966,NaN,NaN,NaN,NaN


In [ ]:
INDIVIDUAL_DATE_OF_BIRTH = INDIVIDUAL_DATE_OF_BIRTH.drop(['DATE_OF_BIRTH','NOTE'],axis=1)
INDIVIDUAL_DATE_OF_BIRTH = INDIVIDUAL_DATE_OF_BIRTH.rename(columns={'DATE':'DATE_OF_BIRTH','YEAR':'YEAR_OF_BIRTH'})
INDIVIDUAL_DATE_OF_BIRTH

,TYPE_OF_DATE,DATE_OF_BIRTH,YEAR_OF_BIRTH,FROM_YEAR,TO_YEAR
0,EXACT,1964-07-17,NaN,NaN,NaN
1,EXACT,1964-01-10,NaN,NaN,NaN
2,EXACT,1960-04-04,NaN,NaN,NaN
3,EXACT,1967-07-04,NaN,NaN,NaN
4,EXACT,1973-09-30,NaN,NaN,NaN
...,...,...,...,...,...
695,EXACT,1988-05-07,NaN,NaN,NaN
696,EXACT,1986-07-27,NaN,NaN,NaN
697,EXACT,1976-10-05,NaN,NaN,NaN
698,EXACT,NaN,1966,NaN,NaN


In [ ]:
df_individuals_all = pd.concat([df_individuals_all,INDIVIDUAL_DATE_OF_BIRTH],axis=1)
df_individuals_all = df_individuals_all.drop(['INDIVIDUAL_DATE_OF_BIRTH'],axis=1)
df_individuals_all.head(2)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,COUNTRY,NOTE,ZIP_CODE,DESIGNATION,NATIONALITY,TYPE_OF_DATE,DATE_OF_BIRTH,YEAR_OF_BIRTH,FROM_YEAR,TO_YEAR
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,UN List,...,None,NaN,NaN,DPRK Ministry of State Security Official,Democratic People's Republic of Korea,EXACT,1964-07-17,NaN,NaN,NaN
1,6908570,1,CHANG,CHANG HA,None,DPRK,KPi.037,2016-11-30,None,UN List,...,None,NaN,NaN,President of the Second Academy of Natural Sci...,Democratic People's Republic of Korea,EXACT,1964-01-10,NaN,NaN,NaN


## 2.7 Clean INDIVIDUAL_ALIAS

In [ ]:
list_temp = pd.DataFrame()
dict_temp = pd.DataFrame()
list_index = []
dict_index = []

# type_list = []
for index, row in df_individuals_all.iterrows():
  row = row['INDIVIDUAL_ALIAS']
  # type_list.append(type(row))
  # print(pd.DataFrame(type_list).rename(columns={0:'type_'}).type_.unique())
  if type(row) == dict:
    temp = pd.DataFrame.from_dict(row, orient="index").T
    dict_temp = dict_temp.append(temp,ignore_index=True)

    dict_index.append(index)
    
    # print('float:{index}'.format(index=index))

  if type(row) == list:
    temp = pd.DataFrame(row)
    temp['QUALITY'] = temp['QUALITY'].apply(lambda x: 'Good' if x=='Low' else x)
    temp = temp.groupby(['QUALITY'])['ALIAS_NAME'].apply(lambda x:', '.join(x) )
    list_temp = list_temp.append(temp,ignore_index=True)
    list_index.append(index)

list_temp.index=list_index
dict_temp.index=dict_index

list_temp.rename(columns={'Good':'ALIAS_NAME'},inplace=True)


In [ ]:
list_temp

,ALIAS_NAME
7,Kim Hak Song
11,"Abd-al-Khaliq al-Huthi, Abd-al-Khaliq Badr-al-..."
12,"Mahad Mohamed Ali Karate, Mahad Warsame Qalley..."
13,"Ayyad Nazmi Salih Khalil, Eyad Nazmi Saleh Kha..."
14,"Ashraf Muhammad Yusif 'Uthman 'Abd-al-Salam, A..."
...,...
695,"Al-Dabachi, Al Ammu, The Uncle, Al-Ahwal, Al D..."
696,"Rahman Salim Milad, al-Bija, Abdurahman Salem ..."
697,"Abu Ibrahim al-Hashimi al-Qurashi, Hajji Abdal..."
698,"Haji Faizullah Noor, Faizullah Noorzai Akhtar ..."


In [ ]:
len(list_temp) + len(dict_temp) == len(df_individuals_all)

True

In [ ]:
dict_temp

,QUALITY,ALIAS_NAME,NOTE,DATE_OF_BIRTH,CITY_OF_BIRTH,COUNTRY_OF_BIRTH
0,None,None,NaN,NaN,NaN,NaN
1,Good,Jang Chang Ha,NaN,NaN,NaN,NaN
2,Good,Jo Chun Ryong,NaN,NaN,NaN,NaN
3,Low,Abu Jihad TNT,NaN,NaN,NaN,NaN
4,Good,Cho Yong Chol,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
684,Good,Mohammad Reza Zahidi; Mohammad Raza Zahedi,NaN,NaN,NaN,NaN
686,Good,Qari Zakir,NaN,NaN,NaN,NaN
691,None,None,NaN,NaN,NaN,NaN
693,Good,Mohammad Bakr Zolqadr; Mohammad Bakr Zolkadr; ...,NaN,NaN,NaN,NaN


In [ ]:
list_temp

,ALIAS_NAME
7,Kim Hak Song
11,"Abd-al-Khaliq al-Huthi, Abd-al-Khaliq Badr-al-..."
12,"Mahad Mohamed Ali Karate, Mahad Warsame Qalley..."
13,"Ayyad Nazmi Salih Khalil, Eyad Nazmi Saleh Kha..."
14,"Ashraf Muhammad Yusif 'Uthman 'Abd-al-Salam, A..."
...,...
695,"Al-Dabachi, Al Ammu, The Uncle, Al-Ahwal, Al D..."
696,"Rahman Salim Milad, al-Bija, Abdurahman Salem ..."
697,"Abu Ibrahim al-Hashimi al-Qurashi, Hajji Abdal..."
698,"Haji Faizullah Noor, Faizullah Noorzai Akhtar ..."


In [ ]:
ALIAS = pd.concat([list_temp,dict_temp]).sort_index()
ALIAS

,ALIAS_NAME,QUALITY,NOTE,DATE_OF_BIRTH,CITY_OF_BIRTH,COUNTRY_OF_BIRTH
0,None,None,NaN,NaN,NaN,NaN
1,Jang Chang Ha,Good,NaN,NaN,NaN,NaN
2,Jo Chun Ryong,Good,NaN,NaN,NaN,NaN
3,Abu Jihad TNT,Low,NaN,NaN,NaN,NaN
4,Cho Yong Chol,Good,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
695,"Al-Dabachi, Al Ammu, The Uncle, Al-Ahwal, Al D...",NaN,NaN,NaN,NaN,NaN
696,"Rahman Salim Milad, al-Bija, Abdurahman Salem ...",NaN,NaN,NaN,NaN,NaN
697,"Abu Ibrahim al-Hashimi al-Qurashi, Hajji Abdal...",NaN,NaN,NaN,NaN,NaN
698,"Haji Faizullah Noor, Faizullah Noorzai Akhtar ...",NaN,NaN,NaN,NaN,NaN


In [ ]:
ALIAS = ALIAS.iloc[:,:2]
# ALIAS.rename(columns={'QUALITY':'ALIAS_QUALITY'},inplace =True)
ALIAS = ALIAS.drop(['QUALITY'],axis=1)
ALIAS

,ALIAS_NAME
0,None
1,Jang Chang Ha
2,Jo Chun Ryong
3,Abu Jihad TNT
4,Cho Yong Chol
...,...
695,"Al-Dabachi, Al Ammu, The Uncle, Al-Ahwal, Al D..."
696,"Rahman Salim Milad, al-Bija, Abdurahman Salem ..."
697,"Abu Ibrahim al-Hashimi al-Qurashi, Hajji Abdal..."
698,"Haji Faizullah Noor, Faizullah Noorzai Akhtar ..."


In [ ]:
df_individuals_all = df_individuals_all.drop(['INDIVIDUAL_ALIAS'],axis=1)
df_individuals_all = pd.concat([df_individuals_all,ALIAS],axis=1)
df_individuals_all.head(2)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,NOTE,ZIP_CODE,DESIGNATION,NATIONALITY,TYPE_OF_DATE,DATE_OF_BIRTH,YEAR_OF_BIRTH,FROM_YEAR,TO_YEAR,ALIAS_NAME
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,UN List,...,NaN,NaN,DPRK Ministry of State Security Official,Democratic People's Republic of Korea,EXACT,1964-07-17,NaN,NaN,NaN,None
1,6908570,1,CHANG,CHANG HA,None,DPRK,KPi.037,2016-11-30,None,UN List,...,NaN,NaN,President of the Second Academy of Natural Sci...,Democratic People's Republic of Korea,EXACT,1964-01-10,NaN,NaN,NaN,Jang Chang Ha


## 2.8 Clean INDIVIDUAL_PLACE_OF_BIRTH

In [ ]:
# pd.DataFrame(df_individuals_all.INDIVIDUAL_PLACE_OF_BIRTH).to_csv('INDIVIDUAL_PLACE_OF_BIRTH.csv')

In [ ]:
# n_type=[]
# for index, row in df_individuals_all.iterrows():
#   row = row['INDIVIDUAL_PLACE_OF_BIRTH']
#   n_type.append(type(row))

# print(pd.DataFrame(n_type).rename(columns={0:'type_'}).type_.unique())

[<class 'NoneType'> <class 'dict'> <class 'list'>]


In [ ]:
# pd.DataFrame(df_individuals_all.INDIVIDUAL_PLACE_OF_BIRTH[23]).iloc[:1,:]

In [ ]:
# test = pd.DataFrame(df_individuals_all.INDIVIDUAL_PLACE_OF_BIRTH[23]).iloc[:1,:]
# test

In [ ]:
# df_individuals_all.INDIVIDUAL_PLACE_OF_BIRTH[137]

In [ ]:
# n_dict = []
# n_list = []
# n_type = []
# for index, row in df_individuals_all.iterrows():
#   row = row['INDIVIDUAL_PLACE_OF_BIRTH']
#   n_type.append(type(row))
  
#   if type(row) == dict:
#   #   n_dict.append(len(row))
#     pass
#   #   print(len(row),index)


#   elif type(row) == list:
#       pass
#   #   n_list.append(len(row))
#   #   print(len(row),index)
#   else:
#       print(index)
#   #   pass

In [ ]:
# type(df_individuals_all.INDIVIDUAL_PLACE_OF_BIRTH[0])  == 'NoneType'

In [ ]:
# df341 = [{'STATE_PROVINCE': 'Logar Province', 'COUNTRY': 'Afghanistan'}, \
#          {'STATE_PROVINCE': 'Kabul Province', 'COUNTRY': 'Afghanistan'},\
#          {'STATE_PROVINCE': 'Logar Province', 'COUNTRY': 'Afghanistan'}]
# len(df341)

In [ ]:
# pd.DataFrame(df341).iloc[:1,:].columns

In [ ]:
none_temp = pd.DataFrame(columns=['COUNTRY','STATE_PROVINCE','CITY','NOTE'])
list_temp = pd.DataFrame()
dict_temp = pd.DataFrame()
list_index = []
dict_index = []
none_index = []

# type_list = []
for index, row in df_individuals_all.iterrows():
  row = row['INDIVIDUAL_PLACE_OF_BIRTH']
  # type_list.append(type(row))
  # print(pd.DataFrame(type_list).rename(columns={0:'type_'}).type_.unique())

  if type(row) == dict:
    temp = pd.DataFrame.from_dict(row, orient="index").T
    dict_temp = dict_temp.append(temp,ignore_index=True)
    dict_index.append(index)
    

  elif type(row) == list:
    # only select first row
    temp = pd.DataFrame(row).iloc[:1,:]
    if len(row)>=2:
      firsr_col = list(temp.columns)[0]
      temp[firsr_col] =  temp[firsr_col].apply(lambda x: str(x) + ' (Other locations cited)')
      list_temp = list_temp.append(temp,ignore_index=True)
    else:
      list_temp = list_temp.append(temp,ignore_index=True)
    list_index.append(index)

  else:
    none_temp = none_temp.append({'COUNTRY':None,'STATE_PROVINCE':None,'CITY':None,'NOTE':None},ignore_index=True)
    none_index.append(index)
  
list_temp.index=list_index
dict_temp.index=dict_index
none_temp.index=none_index

In [ ]:
len(none_temp)+len(list_temp)+len(dict_temp) == len(df_individuals_all)

True

In [ ]:
dict_temp.head(2)

,STATE_PROVINCE,COUNTRY,CITY,NOTE
3,Rio Claro,Trinidad and Tobago,NaN,NaN
12,Xararadheere,Somalia,NaN,NaN


In [ ]:
list_temp.head(5)

,CITY,STATE_PROVINCE,COUNTRY
22,Surkh Rod District (Other locations cited),Nangarhar Province,Afghanistan
26,"Mirmandaw village, Nahr-e Saraj District (Othe...",Helmand Province,Afghanistan
29,NaN,Nangarhar Province (Other locations cited),Afghanistan
37,Korleko-Lombok Timur (Other locations cited),NaN,Indonesia
42,Ndele (Other locations cited),NaN,Central African Republic


In [ ]:
none_temp.head(2)

,COUNTRY,STATE_PROVINCE,CITY,NOTE
0,None,None,None,None
1,None,None,None,None


In [ ]:
INDIVIDUAL_PLACE_OF_BIRTH = pd.concat([dict_temp,list_temp,none_temp]).sort_index()
INDIVIDUAL_PLACE_OF_BIRTH.head(2)

,STATE_PROVINCE,COUNTRY,CITY,NOTE
0,None,None,None,None
1,None,None,None,None


In [ ]:
INDIVIDUAL_PLACE_OF_BIRTH.rename(columns={'COUNTRY':'COUNTRY_OF_BIRTH',
                                          'STATE_PROVINCE':'STATE_PROVINCE_OF_BIRTH',
                                          'CITY':'CITY_OF_BIRTH',
                                          'NOTE':'NOTE_OF_BIRTH'},inplace=True)
INDIVIDUAL_PLACE_OF_BIRTH.head(1)

,STATE_PROVINCE_OF_BIRTH,COUNTRY_OF_BIRTH,CITY_OF_BIRTH,NOTE_OF_BIRTH
0,None,None,None,None


In [ ]:
df_individuals_all = df_individuals_all.drop(['INDIVIDUAL_PLACE_OF_BIRTH'],axis=1)
df_individuals_all = pd.concat([df_individuals_all, INDIVIDUAL_PLACE_OF_BIRTH],axis=1)
df_individuals_all.head(1)

,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE,...,TYPE_OF_DATE,DATE_OF_BIRTH,YEAR_OF_BIRTH,FROM_YEAR,TO_YEAR,ALIAS_NAME,STATE_PROVINCE_OF_BIRTH,COUNTRY_OF_BIRTH,CITY_OF_BIRTH,NOTE_OF_BIRTH
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,UN List,...,EXACT,1964-07-17,NaN,NaN,NaN,None,None,None,None,None


## 2.9 Clean TITLE

In [ ]:
df_individuals_all['TITLE'] = df_individuals_all['TITLE'].apply(lambda x: None if str(x)=='nan' else \
                                                                str(x).replace("{'VALUE':","").replace("}","").\
                                                                replace("[","").replace("]",""))
df_individuals_all['TITLE'] = df_individuals_all['TITLE'].apply(lambda x: None if x==None else x.replace("'",""))

## 2.10 Clean INDIVIDUAL_DOCUMENT

In [ ]:
df_individuals_all = df_individuals_all.drop(['INDIVIDUAL_DOCUMENT'],axis=1)

## 2.11 Check Duplicate DATAID

In [ ]:
len(df_individuals_all.DATAID.unique()) == len (df_individuals_all)

True

In [ ]:
freq = pd.DataFrame(df_individuals_all.isnull().sum()).rename(columns={0:'freq'})

In [ ]:
freq

,freq
DATAID,0
VERSIONNUM,0
FIRST_NAME,0
SECOND_NAME,9
THIRD_NAME,359
UN_LIST_TYPE,0
REFERENCE_NUMBER,0
LISTED_ON,0
COMMENTS1,88
LIST_TYPE,0


In [ ]:
df_individuals_all.dropna(
          axis=1, # axis=1 delete col
          how='all', # how=all delete col if this all col contain null
          inplace=True # inplace=True replace
          )


In [ ]:
df_individuals_all = df_individuals_all.drop(['NAME_ORIGINAL_SCRIPT'],axis=1)

#Finally

In [ ]:
# save to local
df_individuals_all.to_csv('data/UN_sanctioned_individuals.csv')